In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
real_news_data = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")
fake_news_data = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")

### Let's EDA first.

In [ ]:
real_news_data.head()

In [ ]:
real_news_data.info()

In [ ]:
# we change the date from str to datetime 
real_news_data.date = pd.to_datetime(real_news_data.date)
real_news_data.info()

#### Null doesn't exist!

In [ ]:
real_news_data.date.value_counts()

In [ ]:
# we know how many subject the data has
real_news_data.subject.value_counts()

### Let's move the fake data

In [ ]:
fake_news_data.head()

In [ ]:
fake_news_data.info()

In [ ]:
# we change the date from str to datetime 
# fake_news_data.date = pd.to_datetime(fake_news_data.date)
fake_news_data.date.value_counts()

In [ ]:
# so we have the iregular format '14-Feb-18'
# but this below code is not good idea...
fake_news_data.date.value_counts().sort_values()[:30]

In [ ]:
fake_news_data.date.value_counts().sort_values()[-15:]

In [ ]:
fake_news_data[fake_news_data.date.str.contains('-')]

In [ ]:
# check the percentage
len(fake_news_data[fake_news_data.date.str.contains('-')]) / len(fake_news_data) * 100

#### OK, We found the date format of the fake data has various format
#### but just only 0.2% 
#### so, we cut it all!

In [ ]:
fake_news_data = fake_news_data[~fake_news_data.date.str.contains('-')]
fake_news_data.date.value_counts()

In [ ]:
fake_news_data.date.value_counts().sort_values()

In [ ]:
fake_news_data.date.value_counts().sort_values()[:30]

In [ ]:
fake_news_data.date.value_counts().sort_index()[-30:]

In [ ]:
fake_news_data = fake_news_data[~fake_news_data.date.str.contains('MSNB')]
fake_news_data.date.value_counts().sort_index()[:30]

### It looks fineascii

In [ ]:
fake_news_data.date = pd.to_datetime(fake_news_data.date)
fake_news_data.info()

### OK! we done

In [ ]:
fake_news_data.subject.value_counts()

In [ ]:
fake_news_data.text.head(30)

### let's move the analyze
### I don't think We have to use the subjet column this time because we can't use this efficiently

In [ ]:
real_news_data['label'] = 1
fake_news_data['label'] = 0

In [ ]:
merge_data = pd.concat([real_news_data, fake_news_data])
merge_data.head()

In [ ]:
merge_data = merge_data.drop('subject', axis=1)
merge_data.head()

#### I get to think the date data is not practical this time...

In [ ]:
# we merge the title and text with blank
merge_data['all_sentence'] =  merge_data.title + ' ' + merge_data.text
merge_data.head(3)

#### OK, for the moment, I prepare the data.
#### I use the TfidfVectorizer and logistic first 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
X = merge_data['all_sentence'].values
y = merge_data.label.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)

In [ ]:
# we have to prepare stopwords so we use storpword list instantly
# we use nltk 
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')

In [ ]:
def tokenizer(text):
    return text.split()


tfidf = TfidfVectorizer(stop_words=stop, tokenizer=tokenizer)

lr_pipeline = Pipeline([('tfidf', tfidf), ('lr', LogisticRegression())])

param_dist = [{
    'lr__C': np.arange(0.01, 10, 0.5)
}]

random_search = RandomizedSearchCV(estimator=lr_pipeline, param_distributions=param_dist, cv=10)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.score(X_train, y_train)

In [ ]:
random_search.score(X_test, y_test)

### I think this is not enough to use mainly. so I plan to keep modifying regulary
for the moment, I stop this!
thank you!
